### Action1
基于高德地图的路径规划  
从指定地点start，到终点end的路径规划  
最优路径定义：  
1）距离最短  
2）时间最短  
输入：start,end  
输出：路径规划，所需的距离、时间  

#### 数据获取

##### 抓取地铁站信息

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [3]:
def get_page_content(request_url):
    header={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
    html = requests.get(request_url, headers=header, timeout=10)
    content = html.text
#     print(content)
    # 通过content创建BS对象
    soup = BeautifulSoup(content, 'html.parser',from_encoding='utf-8')
    return soup

In [4]:
request_url = 'https://ditie.mapbar.com/shanghai_line/'
soup = get_page_content(request_url)

C:\Users\80566\Coding\anaconda3\lib\site-packages\bs4\__init__.py:220: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [6]:
df = pd.DataFrame(columns=['name', 'line'])
subways = soup.find_all('div', class_='station')
for subway in subways:
#     print(subway)
    # 线路名
    route_name = subway.find('strong', class_='bolder').text
    routes = subway.find('ul')
    routes = routes.find_all('a')
    for route in routes:
        temp = {'name':route.text, 'line':route_name}
        df = df.append(temp, ignore_index=True)
df['city'] = '上海'
# print(df)
df.to_excel('./subways.xlsx', index=False)

##### 根据地铁站信息查询，得到经纬度

In [5]:
def get_location(place, city):
    # 获取查询
    header={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
    request_url = 'https://restapi.amap.com/v3/geocode/geo?key=1eb751daa6e3fd63fbc3737a9433fdd7&address='+ place + '&city=' + city
    data = requests.get(request_url, headers=header)
#     data.endoding= 'utf-8'
#     print(request_url)
    data = data.text
#     print(data)
    
    # 匹配经纬度
    pattern = 'location":"(.*?),(.*?)"'
    result = re.findall(pattern, data) #形如[('121.513870', '31.303558')]
    return result[0][0], result[0][1]

In [42]:
df = pd.read_excel('./subways.xlsx')
df['longitude'], df['latitude'] = None, None
for index,row in df.iterrows():
    try:
        longitude, latitude = get_location(row['name'][:-1]+'地铁站', row['city'])
    except:
        longitude, latitude = get_location(row['name'][:-1], row['city'])
    df.iloc[index]['longitude'] = longitude
    df.iloc[index]['latitude'] = latitude
df.to_excel('./subway_location.xlsx', index=False)

#### 路径规划

In [4]:
import requests
import pandas as pd
import re
from collections import defaultdict
import route_api
import pickle

In [2]:
# 数据加载
data = pd.read_excel('./subway_location.xlsx')
data.head()

,name,line,city,longitude,latitude
0,龙阳路站,磁悬浮,上海,121.557855,31.202480
1,浦东国际机场站,磁悬浮,上海,121.805591,31.150958
2,莘庄站,地铁1号线,上海,121.385379,31.111193
3,外环路站,地铁1号线,上海,121.393020,31.120899
4,莲花路站,地铁1号线,上海,121.402132,31.130432


##### 得到邻接矩阵

In [10]:
import route_api # 详见文件route_api.py
from collections import defaultdict

graph = defaultdict(dict)

for i in range(data.shape[0]-1):
    site1 = data.iloc[i]['line']
    site2 = data.iloc[i+1]['line']
    if site1 == site2:
        station1 = data.iloc[i]['name']
        station2 = data.iloc[i+1]['name']
        longitude1, latitude1 = data.iloc[i]['longitude'], data.iloc[i]['latitude']
        longitude2, latitude2 = data.iloc[i+1]['longitude'], data.iloc[i+1]['latitude']
        distance = route_api.compute_distance(longitude1, latitude1, longitude2, latitude2)
        graph[station1][station2] = distance
        graph[station2][station1] = distance
# print(graph)

In [24]:
import pickle
# output = open('graph.pkl','wb')
# pickle.dump(graph, output)
file = open('graph.pkl','rb')
graph = pickle.load(file)

##### 进行路径规划

In [67]:
def find_lowest_cost_node(costs): # 找到U中值最小的节点
    # 初始化数据
    lowest_cost = float('inf')
    lowest_cost_node = None
    for node in costs:
        if node not in processed:
            if costs[node] < lowest_cost:
                lowest_cost = costs[node]
                lowest_cost_node = node          
    return lowest_cost_node

def find_shortest_path(): # 找到最短路径
    node = end
    shortest_path = [end]
    while parents[node] != start:
        shortest_path.append(parents[node])
        node = parents[node]
    shortest_path.append(start)
    shortest_path.reverse()
    return shortest_path

def dijkstra(): # 计算图中各个点距离起点的最短路程
    node = find_lowest_cost_node(costs)
    while node:
        cost = costs[node]
        neighs = graph[node].keys()
        for neigh in neighs:
            new_cost = cost + float(graph[node][neigh])
            if neigh not in costs or new_cost<costs[neigh]:
                costs[neigh] = new_cost
                parents[neigh] = node
        processed.append(node)
        node = find_lowest_cost_node(costs)
    shortest_path = find_shortest_path()
    return shortest_path      

In [71]:
start = '江湾体育场站'
end = '徐家汇站'

In [99]:
costs = {}
parents = {}
parents[end] = None
costs[end] = float('inf')
processed = [] #S
for node in graph[start].keys():
    costs[node] = float(graph[start][node])
    parents[node] = start
    
shortest_path = dijkstra()
print('从{}到{}的最短路径为{}'.format(start, end, shortest_path))

从江湾体育场站到徐家汇站的最短路径为['江湾体育场站', '五角场站', '国权路站', '同济大学站', '四平路站', '邮电新村站', '海伦路站', '四川北路站', '天潼路站', '曲阜路站', '汉中路站', '南京西路站', '陕西南路站', '常熟路站', '衡山路站', '徐家汇站']
